# Tutorial for Small FOV Instruments

In this tutorial we combine our skymaps with galaxy catalogs to get a list of galaxies for individual pointings. A note is made that this is only possible with 3D skymaps which are provided for combact binary merger candidate events.

We begin by importing the necessary packages as done previously. We will also download the 2MASS Redshift Survey galaxy catalog using VizieR.

In [1]:
# -- For Google Colab
#! pip install -q "astropy==3.2.3" "astroquery==0.3.10" "healpy==1.12.9" "matplotlib==3.1.2" "scipy==1.4.1"

import healpy as hp # for working with HEALPix files
import numpy as np # needed for vector operations
from matplotlib import pyplot as plt # plotting skymaps
from scipy.stats import norm # probability functions

In [2]:
from astropy.utils.data import download_file
url = ('https://dcc.ligo.org/public/0146/G1701985/001/LALInference_v2.fits.gz')
# This is the publication LALInference localization
filename = download_file(url, cache=True)

We read in the probability, distmu, distsigma, and distnorm.

In [3]:
prob, distmu, distsigma, distnorm = hp.read_map(filename, field=range(4))

NSIDE = 1024
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING


In [4]:
npix = len(prob)
nside = hp.npix2nside(npix)
npix, nside

(12582912, 1024)

In [5]:
# Area per pixel in steradians
pixarea = hp.nside2pixarea(nside)

In [6]:
from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1 # This gets the complete catalog
cat1, = Vizier.get_catalogs('J/ApJS/199/26/table3') # Downloading the 2MRS Galaxy Catalog

According to Tully(2015), the 2MRS luminosity function is well fit by a Schechter function with a cutoff absolute magnitude of $M_k^* = -23.55$ and a power-law index of $\alpha_K = -1$. We find the maximum absolute magnitude $M_k^{\text{max}}$ for a completeness fraction of 0.5.

In [7]:
from scipy.special import gammaincinv
completeness = 0.5
alpha = -1.0
MK_star = -23.55
MK_max = MK_star + 2.5*np.log10(gammaincinv(alpha + 2, completeness))
MK_max

-23.947936347387156

Now, we select only galaxies with positive redshifts and absolute magnitudes greater than $M_k^{\text{max}}$.

In [8]:
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Column
import astropy.units as u
import astropy.constants as c
z = (u.Quantity(cat1['cz'])/c.c).to(u.dimensionless_unscaled)
MK = cat1['Ktmag']-cosmo.distmod(z)
keep = (z > 0) & (MK < MK_max)
cat1 = cat1[keep]
z = z[keep]

/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/astropy/cosmology/core.py:1251: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  args=self._inv_efunc_scalar_args)[0]
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/astropy/cosmology/core.py:1447: RuntimeWarning: divide by zero encountered in log10
  val = 5. * np.log10(abs(self.luminosity_distance(z).value)) + 25.0
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/astropy/table/column.py:991: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)


Now, we calculate the luminosity distance and HEALPix index of each galaxy.

In [9]:
r = cosmo.luminosity_distance(z).to('Mpc').value
theta = 0.5*np.pi - cat1['DEJ2000'].to('rad').value
phi = cat1['RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)

We find the probability density per unit volume at the position of each galaxy.

In [10]:
dp_dV = prob[ipix] * distnorm[ipix] * norm(distmu[ipix], distsigma[ipix]).pdf(r)/pixarea

Finally, we sort the galaxies by descending probability density and take the top 50.

In [11]:
top50 = cat1[np.flipud(np.argsort(dp_dV))][:50]
top50['RAJ2000', 'DEJ2000', 'Ktmag']

RAJ2000,DEJ2000,Ktmag
deg,deg,mag
float64,float64,float32
197.01802,-23.79687,9.226
199.52112,-26.83722,7.265
199.95851,-27.41009,7.100
196.89058,-24.00856,9.047
202.30795,-33.17384,8.612
200.44267,-27.43052,7.131
194.36627,-19.69128,8.819
202.86783,-34.79443,8.500


The coordinates of the first galaxy above are (197.01802, -23.79687). A pointing in this direction would likely have captured the true host galaxy of GW170817 which is (197.45, -23.38).

Now, we attempt a similar down-selection but with a different galaxy catalog: the Galaxy List for the Advanced Detector Era.

In [12]:
catalog_list = Vizier.find_catalogs('GLADE')
{k:v.description for k,v in catalog_list.items()}

{'VII/275': 'GLADE catalog (Dalya+, 2016)',
 'VII/281': 'GLADE v2.3 catalog (Dalya+, 2018)'}

In [ ]:
catalogs = Vizier.get_catalogs(catalog_list.keys())
catalogs

In [ ]:
Vizier.ROW_LIMIT = 50000
# Note, the GLADE catalog is 1,918,147 rows long thus we will get a memory error if we set the row limit to -1
cat2, = Vizier.get_catalogs('VII/275/glade1') # Downloading the GLADE catalog (Galaxy List for the Advanced Detector Era)

According to Gehrels et al(2016), the GLADE luminosity function is well fit by a Schechter function with a cutoff absolute magnitude of $M_k^* = -20.47$ and a power-law index of $\alpha_K = -1.07$. We find the maximum absolute magnitude $M_k^{\text{max}}$ for a completeness fraction of 0.5.

In [ ]:
completeness = 0.5
alpha = -1.07
MK_star = -20.47
MK_max = MK_star + 2.5*np.log10(gammaincinv(alpha + 2, completeness))
MK_max

In [ ]:
dist = u.Quantity(cat2['Dist']) # Distance in Mpc
z = (u.Quantity(cat2['zph2MPZ'])).to(u.dimensionless_unscaled)
MK = cat2['Kmag2']-cosmo.distmod(z)
keep = (z > 0) & (MK < MK_max)
cat2 = cat2[keep]
dist = dist[keep]

In [ ]:
r = dist.value
theta = 0.5*np.pi - cat2['DEJ2000'].to('rad').value
phi = cat2['RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)

In [ ]:
dp_dV = prob[ipix] * distnorm[ipix] * norm(distmu[ipix], distsigma[ipix]).pdf(r)/pixarea

In [ ]:
top50 = cat2[np.flipud(np.argsort(dp_dV))][:50]
top50['RAJ2000', 'DEJ2000', 'Kmag2']

We can use many different algorithms to prioritize galaxies. Here is an outline used by the Las Cumbres Observatory for following-up on GW170817.

Step 1: Compute the probability the source being at a distance $D$ for a certain right ascension and declination. Compute the location score of a galaxy: $S_{\text{loc}} = p_{\text{pos}} \times p_{\text{dist}}$.

Step 2: Using the $B$-band magnitude and distance provided in the catalog, calculate the $B$-band luminosity of the galaxy: $L_B$.

Step 3: Assuming a limiting magnitude for exposures, $m_{\text{lim}}$, convert to a limiting luminosity at the distance of each galaxy, $L_{\text{lim}}$. Assign the galaxy a score: $S_{\text{lum}} = \frac{L_B}{\Sigma L_B}$

Step 4: Define the likely counterpart magnitude range, $M_{\text{KN,min}} - M_{\text{KN,max}}$ and convert those magnitudes to luminosities.

Step 5: Give each galaxy a detection likelihood score: $S_\text{det} = \frac{L_{\text{KN,max}} - L_{\text{lim}}}{L_{\text{KN,max}} - L_{\text{KN,min}}}$.